# 训练神经网络

本章中，我们要学习更新spaCy的统计模型使其能够为特定的使用场景做出定制化。一个例子是我们想要在网络上的评论中抽取一种新的实体。我们将会学到如何从头训练自己的模型，了解模型训练的基本工作原理，以及一些技巧使得我们自己的定制化自然语言处理项目能够更加成功。

## 模型的训练和更新

#### 为什么要更新模型？

- 对特定领域有更好的结果
- 对特定问题学习特定的分类类别
- 对文本分类非常必要
- 对命名实体识别非常有用
- 对词性标注和依存关系识别不是很关键

我们介绍 *如何* 更新模型之前先花一点时间问问我们自己：为什么我们想要用我 们自己的例子来更新模型？我们是不是只用预训练好的流程就可以了？

统计模型可以基于相应的训练数据来做预测。

给模型更多我们自己特定领域的数据通常可以让模型预测更准确。

我们经常需要预测一些和我们特定问题相关的类别，所以模型也需要学习。

更新模型对文本分类十分必要，对实体识别也非常有用，但对词性标注和依存句法分析就没 有那么关键了。


#### 如何训练(1)

spaCy 支持用更多的例子来更新现有模型训练新模型。如果我们不是用一个预训练 模型做初始模型的话，我们首先需要随机化所有的权重。

然后 spaCy 调用`nlp.update`，这个方法使用当前权重来预测一批次的例子。

模型然后把预测结果和正确答案做比较，决定下一步应该如何改变模型权重来使得下一次的 预测结果表现更好。

最后我们对当前权重做出微调，然后在下一个批次的例子上做预测。

spaCy 然后对数据中的每一批例子调用`nlp.update`。在训练过程中，我们通常要多次遍历 数据进行训练，直到模型不再变得更好。

1. **初始化**模型权重使之变为随机值
2. **预测**几个例子，看看当前权重的表现
3. **比较**预测结果和真实标注的标签
4. **计算**如何调整权重来改善预测结果
5. **微调**模型权重
6. 重复步骤 2。

#### 如何训练(2)

我们来看看模型训练的过程。

训练数据是我们想要用来更新模型的输入例子。

文本应该是一个句子、一个段落或者更长的文档。要得到最好的结果，这些文本应该尽可能 与模型将要在生产环境中预测的文本格式相似。

标签是我们想让模型预测的结果。标签可以是一个文本类别，或者是一个实体的跨度 span 及其类别。

梯度决定了我们应该如何微调模型来减少当前的错误。我们通过比较模型预测标签和真实标 签来计算梯度。

训练结束后，我们就可以保存更新过的模型，在实际应用中使用它了。

![训练流程图表](https://course.spacy.io/training_zh.png)

- **训练数据：** 例子及其标注
- **文本：** 输入文本，模型会在这些文本上做标签预测。
- **标签：** 模型需要预测的标签
- **梯度：** 权重如何微调

##### 举例：训练实体识别器

我们来看一个例子，专注于一个特定的组件：实体识别器。

实体识别器读入一个文档，预测其中的短语及其 *语境中* 的标签。这意味着训练数据需要 有文本、包含的实体以及实体的标签。

实体之间不能重叠，所以一个词符只能属于一个实体。

最简单的方法就是给模型输入一段文本和一个字符位置的列表。spaCy 可以通过标准的含有 标注为`doc.ents`的实体`Doc`对象来进行更新。比如"iPhone X"是一个电子产品，是从字 符 0 开始到字符 8 结束的。

让模型知道哪些词 *并不是* 实体也是非常重要的。

这种情况下，span 标注的列表会是空的。

我们的目标是教会模型在相似语境中识别出新的实体，就算这些实体并没有在训练数据中出 现过。

- 实体识别器可以对词语和短语做基于语境的实体识别
- 每一个词符只能是某一个实体的一部分
- 训练例子需要带有语境

```python
doc = nlp("iPhone X就要来了")
doc.ents = [Span(doc, 0, 8, label="GADGET")]
```

- 并非实体的文本部分也非常重要

```python
doc = nlp("我急需一部新手机，给点建议吧！")
doc.ents = []
```

- **目标：**让模型学会泛化


##### 训练数据

训练数据告诉模型我们想要预测什么。预测目标可能是想要识别的文本和命名实体 ，或者是词符及其正确的词性标签，或是任何模型可以预测的结果。

要更新一个现有的模型，我们可以先试试几百到几千数据量的例子。

要训练一个新的类别我们可能需要百万级别的训练数据。

spaCy 的训练好的中文流程是在 200 万个词汇的语料上训练的，这些文本都已经标注了词 性标签、依存关系和命名实体。

训练数据通常是由训练师手动标注文本创建的。

数据标注是一项繁重的任务，但也可以半自动化进行，比如用 spaCy 的模板匹配 器`Matcher`。

- 一系列我们希望模型在语境中做出预测的例子

- 更新**现有模型**：需要几百到几千个例子

- 训练

  新的类别

  ：需要几千乃至几百万个例子

  - spaCy 的中文模型是在 200 万个词语的语料规模上训练的

- 通常需要由标注师人工标注数据

- 也可以是半自动的，比如用 spaCy 的模板匹配器`Matcher`！


##### 训练数据 vs 测试数据

训练模型中的重要一环是了解模型的表现，模型学习的方向是否正确。方法是让模 型在一些训练中 *尚未* 见过的数据上进行预测，然后对比我们已经知道的正确答案。因此 在训练数据之外，我们还需要测试数据，也被称作为开发数据。

测试数据用来计算模型的准确度。举个例子，一个准确度为 90%的模型意味着模型在测试数 据上的预测结果中 90%都是正确的。

这意味着测试数据需要能够代表模型在生产环境中会遇到的数据，否则准确度也将失去意义 ，因为这不能告诉我们模型 *真正* 表现得有多好。

- **训练数据：**用来更新模型
- 测试数据：
  - 模型在训练过程中未见到的数据
  - 用来计算模型的准确度
  - 代表了模型在生产环境中会遇到的真实数据


##### 生成训练语料(1)

spaCy 可以用与其创建的`Doc`对象相同格式的数据来更新。我们已经在第二章中学 到了很多如何创建`Doc`和`Span`对象的知识。

在这个例子中，我们为语料创建了两个`Doc`对象：一个中含有一个实体而另一个不含有实 体。要为`Doc`设置实体，我们需要把`Span`加入到`doc.ents`中。

当然了，我们还需要更多的训练数据来有效地训练出一个可以泛化和在语境中预测类似实体 的模型。根据任务的不同，我们通常希望有至少几百到上千个有代表性的数据。

```python
import spacy
nlp = spacy.blank("zh")
# 创建一个含有实体span的Doc
doc1 = nlp("iPhone X就要来了")
doc1.ents = [Span(doc1, 0, 8, label="GADGET")]
# 创建另一个没有实体span的Doc
doc2 = nlp("我急需一部新手机，给点建议吧！")
docs = [doc1, doc2]  # 以此类推...
```


##### 生成训练语料(2)

如前面所提到的，我们不仅需要数据来训练模型，还需要在模型训练中未见过的数 据上面测试模型准确度。我们通常对数据随机排序，然后把数据分成两份：一份作为训练数 据，一份作为测试数据。这里我们只是简单的 50/50 对半分。

- 将数据分割成两份:
  - **训练数据：**用来更新模型
  - **开发数据：**用来测试模型

```python
random.shuffle(docs)
train_docs = docs[:len(docs) // 2]
dev_docs = docs[len(docs) // 2:]
```


##### 生成训练语料(3)

我们一般希望可以将训练和测试数据保存为硬盘上的文件，这样我们就可以读入到 spaCy 的训练流程中。

`DocBin`是用来有效存储和序列化`Doc`对象的容器。我们可以用一个`Doc`对象的列表来初 始化它，然后调用 `to_disk`方法将其存储为一个二进制文件。这些文件我们一般使 用`.spacy`作为后缀。

相比起其它如`pickle`的二进制序列化规制，我们的`DocBin`会更加快，生成的文件更小， 因为其仅对于共享的词汇表仅存储一次。关于`DocBin`运作的更多相关信息可以查 询[文档](https://spacy.io/api/docbin).

- `DocBin`：用来有效存储`Doc`对象的容器
- 可以保存为二进制文件
- 二进制文件可以用来训练模型

```python
# 创建和保存一系列的训练文档
train_docbin = DocBin(docs=train_docs)
train_docbin.to_disk("./train.spacy")
# 创建和保存一系列的测试文档
dev_docbin = DocBin(docs=dev_docs)
dev_docbin.to_disk("./dev.spacy")
```


##### Tips：数据转换

有时候我们的训练和测试数据可能已经是常见的格式 - 比如 CoNLL 或者 IOB. spaCy 的`convert`命令可以自动将这些文件转换为 spaCy 的二进制格式。它也可以转换 spaCy v2 中使用旧格式的 JSON 文件。

- `spacy convert`可以将语料转换为常见的格式
- 支持`.conll`, `.conllu`, `.iob`以及 spaCy 之前的 JSON 格式。

```bash
$ python -m spacy convert ./train.gold.conll ./corpus
```

是时候开始准备一些训练语料了。我们来看看几个例子，为一个新的实体类别创建 一个小的训练数据集。



##### 练习：训练和评估数据

要训练一个模型，我们通常需要训练数据 *和* 用来评估模型的开发数据。这个评估数据是用来做什么的？

1. 如果训练数据不够的时候用来提供更多的训练例子。
2. 在未见过的数据上做预测并计算准确度分数。
3. 没有标注数据时来定义训练的例子。

回答：

1. **不正确。** 

   训练过程中，模型只能由训练数据来进行更新。开发数据只是用来将模型在未见过的数据上预测的结果与真实标注做对比， 来评估模型表现的。准确度分数由此计算而来。

2. **正确！** 

   开发数据只是用来将模型在未见过的数据上预测的结果与真实标注做对比， 来评估模型表现的。准确度分数由此计算而来。

3. **不正确。** 

   开发数据只是用来将模型在未见过的数据上预测的结果与真实标注做对比， 来评估模型表现的。准确度分数由此计算而来。


#### 练习：创建训练数据集

spaCy的基于规则的`Matcher`可以很好地被用来快速创建一些命名实体模型的训练数据。变量 `TEXTS`中存储着句子的列表，我们可以将其打印出来做检查。我们想要找到所有对应不同iPhone 型号的文本，所以我们可以创建一些训练数据来教会模型把它们识别为电子产品`"GADGET"`。

- 编写一个模板，含有两个词符且它们的小写形式可以匹配到`"iphone"`和`"x"`。
- 编写一个模板，含有两个词符，第一个词符的小写形式匹配到`"iphone"`，第二个词符用`"?"` 运算符匹配到一个数字。


In [2]:
import json
import spacy
from spacy.matcher import Matcher
from spacy.tokens import Span

with open("iphone.json", encoding="utf8") as f:
    TEXTS = json.loads(f.read())

nlp = spacy.blank("zh")
matcher = Matcher(nlp.vocab)

# 两个词符，其小写形式匹配到"iphone"和"x"上
pattern1 = [{"LOWER": "iphone"}, {"LOWER": "x"}]

# 词符的小写形式匹配到"iphone"和一个数字上
pattern2 = [{"LOWER": "iphone"}, {"IS_DIGIT": True}]

# 把模板加入到matcher中，并用匹配到的实体创建docs
matcher.add("GADGET", [pattern1, pattern2])
docs = []
for doc in nlp.pipe(TEXTS):
    matches = matcher(doc)
    spans = [Span(doc, start, end, label=match_id) for match_id, start, end in matches]
    print(spans)
    doc.ents = spans
    docs.append(doc)

[]
[]
[]
[]
[]
[]


#### 练习：创建训练数据（2）

在为我们的语料创建数据之后，我们需要将其存放在一个后缀为`.spacy`的文件中。可以参见上一个例子中的代码。

- 使用`docs`的列表初始化`DocBin`。
- 将`DocBin`存储到一个名为`train.spacy`的文件中。


In [3]:
import json
import spacy
from spacy.matcher import Matcher
from spacy.tokens import Span, DocBin

with open("iphone.json", encoding="utf8") as f:
    TEXTS = json.loads(f.read())

nlp = spacy.blank("zh")
matcher = Matcher(nlp.vocab)
# 将pattern加入mattcher中
pattern1 = [{"LOWER": "iphone"}, {"LOWER": "x"}]
pattern2 = [{"LOWER": "iphone"}, {"IS_DIGIT": True}]
matcher.add("GADGET", [pattern1, pattern2])
docs = []
for doc in nlp.pipe(TEXTS):
    matches = matcher(doc)
    spans = [Span(doc, start, end, label=match_id) for match_id, start, end in matches]
    doc.ents = spans
    docs.append(doc)

doc_bin = DocBin(docs=docs)
doc_bin.to_disk("./train.spacy")

## 配置和运行训练流程

我们现在已经学会了如何创建训练数据，我们来看看如何配置和训练流程。

本节课中我们会 学习到spaCy的训练配置系统，如何生成我们自己的训练配置，如果使用CLI来训练模型，以及如何 在训练结束后测试我们的流程。

#### 训练配置(1)

- 所有设定的**唯一真理来源**
- 通常被叫做`config.cfg`
- 定义了如何初始化`nlp`对象
- 包含了关于流程组件和模型实现的所有设定
- 配置了训练过程和超参数
- 使我们的训练过程可复现

spaCy使用的配置文件通常被叫做`config.cfg`，是所有设定的“唯一真理来源”。这个 配置文件决定了如何初始化`nlp`对象，哪些流程组件被添加，以及如何配置组件内部的模型实现。 配置文件还包含了训练过程的所有设定，包括如何读取数据和超参数等。

由此我们再不需要在命令行提供大量的的参数，或是在代码中记着定义每一个设定。我们只需要把 配置文件传给spaCy的训练指令即可。

配置文件也帮助我们可以更好复现训练过程：所有的设定都在同一个地方，流程训练一目了然。 我们甚至可以将配置文件放到Git仓库中，加入版本控制分享给其他人，这样其他人也可以 用同样的设定训练同样的流程。


#### 训练配置(2)

这是从训练一个命名实体识别器的流程配置文件中摘取的片段。配置文件分为几个部分， 嵌套部分用一个点来定义。比如，`[components.ner.model]`定义了命名实体识别器 的模型实现的设定。

配置文件也可以用`@`标记来引用Python函数。比如，分词器定义了一个注册过的分词函数。我们 可以用它来定制化`nlp`对象和训练的不同部分 - 从嵌入我们自己的分词器到实现我们自己的模型 架构。但是我们现在先不用担心 - 本章节中我们只是简单使用spaCy提供的开箱可用的默认配置。

```ini
[nlp]
lang = "zh"
pipeline = ["tok2vec", "ner"]
batch_size = 1000
[nlp.tokenizer]
@tokenizers = "spacy.zh.ChineseTokenizer"
segmenter = "char"
[components]
[components.ner]
factory = "ner"
[components.ner.model]
@architectures = "spacy.TransitionBasedParser.v2"
hidden_width = 64
# 以此类推
```

#### 生成一个配置文件

当然了，我们往往不需要手写配置文件，很多情况下我们甚至不需要定制化配置文件。 spaCy会自动帮我们生成。

文档中的快速上手插件可以交互式地帮我们生成配置文件，让我们选择需要的语言和流程组件以及 可选的硬件和优化设定。

另外，我们也可以使用spaCy内建的`init config`命令。该命令的第一个参数是输出文件，我们通常 起名为`config.cfg`. 参数`--lang`定义了流程的语言类，比如`zh`就是中文。`--pipeline`参数 让我们指定一个或多个用逗号分隔的流程组件来加入流程之中。这个例子中，我们创建了一个配置文件， 含有一个命名实体识别的流程组件。

- spaCy可以自动生成一个默认的配置文件
- 文档中有可交互的[快速上手插件](https://spacy.io/usage/training#quickstart)
- 作用于CLI的[`init config`](https://spacy.io/api/cli#init-config)命令

In [4]:
!nvidia-smi

Fri Jan 26 15:19:37 2024       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 551.23                 Driver Version: 551.23         CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                     TCC/WDDM  | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce RTX 3070 Ti   WDDM  |   00000000:01:00.0  On |                  N/A |
|  0%   45C    P8             15W /  290W |    1459MiB /   8192MiB |     20%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [5]:
!python -m spacy init config ./config.cfg --lang zh --pipeline ner --gpu

ℹ Generated config template specific for your use case
- Language: zh
- Pipeline: ner
- Optimize for: efficiency
- Hardware: GPU
- Transformer: bert-base-chinese
✔ Auto-filled config with all values
✔ Saved config
config.cfg
You can now add your data and train your pipeline:
python -m spacy train config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy


- `init config`: 要运行的命令
- `config.cfg`: 生成的配置文档的输出路径
- `--lang`: 流程的语言类，比如中文是 `zh`
- `--pipeline`: 用逗号分隔的流程组件名称

#### 训练流程(1)

要训练一个流程，我们需要的只是`config.cfg`和训练与测试数据。这些数据都是在之前练习中 我们见到过的`.spacy`文件。

`spaCy train`的第一个参数是配置文件的路径。`--output`参数可以指定保存最终训练好的流程的输出路径。

我们还可以在命令行中覆盖不同的配置设定。在这个例子里面，我们用`train.spacy`文件的路径 覆盖了`paths.train`，用`dev.spacy`文件的路径覆盖了`paths.dev`.

- 我们需要的只是`config.cfg`和训练与测试数据
- 配置的设定可以在命令行中被覆盖


In [15]:
!python -m spacy train ./config.cfg --output ./output --paths.train train.spacy --paths.dev dev.spacy --gpu-id 0

Traceback (most recent call last):
  File "<frozen runpy>", line 189, in _run_module_as_main
  File "<frozen runpy>", line 148, in _get_module_details
  File "<frozen runpy>", line 112, in _get_module_details
  File "c:\ProgramData\miniconda3\envs\nlp\Lib\site-packages\spacy\__init__.py", line 6, in <module>
    from .errors import setup_default_warnings
  File "c:\ProgramData\miniconda3\envs\nlp\Lib\site-packages\spacy\errors.py", line 3, in <module>
    from .compat import Literal
  File "c:\ProgramData\miniconda3\envs\nlp\Lib\site-packages\spacy\compat.py", line 4, in <module>
    from thinc.util import copy_array
  File "c:\ProgramData\miniconda3\envs\nlp\Lib\site-packages\thinc\__init__.py", line 5, in <module>
    from .config import registry
  File "c:\ProgramData\miniconda3\envs\nlp\Lib\site-packages\thinc\config.py", line 5, in <module>
    from .types import Decorator
  File "c:\ProgramData\miniconda3\envs\nlp\Lib\site-packages\thinc\types.py", line 25, in <module>
    from .

#### 训练流程(2)

这是训练过程中和结束时我们会看到的一个屏幕输出的例子。我们还记得之前提到过，我们 通常希望在训练过程中遍历数据多次。每一次遍历数据被叫做一个"epoch"。这就是表中的第一列。

每一个epoch中，spaCy会在每200个数据后输出准确度分数。这是第二列中显示的步骤。我们可以 在配置文件中修改这个频率。每一行显示了训练中的这一步模型损失和计算得到的准确度分数。

我们要留意的最有趣的分数时最后一列的合成分数。这反映了我们的模型在测试数据中预测正确的准确度。

训练过程会一直进行直到模型没有进一步的改进空间了，这时程序就会自动退出。

```
============================ Training pipeline ============================
ℹ Pipeline: ['tok2vec', 'ner']
ℹ Initial learn rate: 0.001
E    #       LOSS TOK2VEC  LOSS NER  ENTS_F  ENTS_P  ENTS_R  SCORE
---  ------  ------------  --------  ------  ------  ------  ------
  0       0          0.00     26.50    0.73    0.39    5.43    0.01
  0     200         33.58    847.68   10.88   44.44    6.20    0.11
  1     400         70.88    267.65   33.50   45.95   26.36    0.33
  2     600         67.56    156.63   45.32   62.16   35.66    0.45
  3     800        138.28    134.12   48.17   74.19   35.66    0.48
  4    1000        177.95    109.77   51.43   66.67   41.86    0.51
  6    1200         94.95     52.13   54.63   67.82   45.74    0.55
  8    1400        126.85     66.19   56.00   65.62   48.84    0.56
 10    1600         38.34     24.16   51.96   70.67   41.09    0.52
 13    1800        105.14     23.23   56.88   69.66   48.06    0.57
✔ Saved pipeline to output directory
/path/to/output/model-last
```


#### 读取已经训练好的流程

训练结束后存储的流程是一个正常的可读取的spaCy流程 - 就像其它spaCy提供的训练好的流程一样， 比如 `zh_core_web_sm`. 最终，最后训练出的流程和最高分的流程都会被存储在输出路径中。

我们可以把路径传给`spacy.load`来读取已经训练好的流程。我们接下来就可以用它来处理和分析文本了。

- 训练后的输出是一个正常的可读取的spaCy流程
  - `model-last`: 最后训练出的流程
  - `model-best`: 表现最好的训练流程
- 用`spacy.load`读取流程

In [ ]:
import spacy
nlp = spacy.load("/path/to/output/model-best")
doc = nlp("iPhone 11 vs iPhone 8: 到底有什么区别？")
print(doc.ents)

#### Tips：将流程打包

为了更方便地部署我们的流程，spaCy提供了一系列趁手的命令来将流程打包成Python包。 `spacy package`读取的参数包括我们生成的流程路径和输出路径，然后生成一个含有我们流程的 Python包。这个Python包是`.tar.gz`格式的文件，可以安装到环境中。

我们还可以在命令中提供可选的名字和版本号，这样我们就可以管理同一个流程的多个不同版本， 比如我们想继续定制化我们的流程或者用更多的数据训练它。

使用这个包和使用其它Python包是一样的。安装完后，我们可以用包名来读取流程。注意spaCy会自动把 语言代码加到名字中，所以我们的流程`my_pipeline`最后就成了`zh_my_pipeline`.

- [`spacy package`](https://spacy.io/api/cli#package): 创建一个包含我们流程的可安装的Python包
- 方便版本控制和部署


In [ ]:
%python -m spacy package /path/to/output/model-best ./packages --name my_pipeline --version 1.0.0
%cd ./packages/zh_my_pipeline-1.0.0
%pip install dist/zh_my_pipeline-1.0.0.tar.gz

安装后读取和使用流程：

In [ ]:
nlp = spacy.load("zh_my_pipeline")

让我们上手来训练我们的第一个流程！我们会练习生成一个命名实体识别器的配置文件， 然后用之前练习中生成的数据来训练这个流程。

#### 练习：训练配置

`config.cfg`文件是使用spaCy训练流程的“唯一真理来源”。下列关于配置的说法哪个是 **错误** 的？

1. 使我们可以配置训练流程和超参数。
2. 帮助实现训练流程的可复现。
3. 会为我们的流程创建一个可安装的Python包。
4. 定义了流程的组件和各自的设定。

回答：

1. **不正确。** 

   配置文件含有训练流程的所有设定，包括超参数。

2. **不正确。** 

   配置文件含有 *所有* 设定，也没有隐藏的默认值，所以可以帮助我们的训练实验更加容易复现。 其他人可以轻松通过相同设定重新跑通我们的实验。

3. **正确！** 

   配置文件含有和训练与流程相关的所有设定，但并不能为流程打包。 要创建可安装的Python包，我们可以使用`spacy package`命令。

4. **不正确。** 

   配置文件中的’[components]‘包含了所有流程组件和各自的设定，包括所使用的模型实现。


#### 练习：生成一个配置文件

[`init config`命令](https://spacy.io/api/cli#init-config) 自动生成一个使用默认设定的训练配置文件。 我们想要训练一个命名实体识别器，所以我们要生成一个含有一个流程组件`ner`的配置文件。 因为我们在本课程中是在Jupyter环境中运行命令，所以加上前缀`!`。 如果是在本地终端中运行则不需要加这个前缀。


##### 第一部分

- 使用spaCy的`init config`命令来自动生成一个中文流程的配置。
- 将配置保存到文件`config.cfg`中。
- 使用`--pipeline`参数指明一个流程组件`ner`。


In [ ]:
!python -m spacy init config ./config.cfg --lang zh --pipeline ner

##### 第二部分

我们来看看spaCy刚刚生成的配置文件！ 我们可以运行下面的命令将配置打印到屏幕上。


In [ ]:
!cat ./config.cfg

#### 练习：使用训练客户端

让我们用前面练习中生成的配置文件和训练语料来训练一个命名实体识别器!

使用[`train`](https://spacy.io/api/cli#train) 命令来调取训练配置文件来训练一个模型。 一个名为`config_gadget.cfg`的文件已经在`exercise/zh`中了， 同时还有一个名为`train_gadget.spacy`的文件包含了一些训练数据，`dec_gadget.spacy`文件包含了测试数据。 因为我们在本课程中是在Jupyter环境中运行命令，所以加上前缀`!`。 如果是在本地终端中运行则不需要加这个前缀。

- 在文件`exercises/zh/config_gadget.cfg`上面运行`train`命令。
- 将训练好的流程保存在`output`文件夹中。
- 传入路径`exercises/zh/train_gadget.spacy` 和 `exercises/zh/dev_gadget.spacy`


In [ ]:
!python -m spacy train ./exercises/zh/config_gadget.cfg --output ./output --paths.train ./exercises/zh/train_gadget.spacy --paths.dev ./exercises/zh/dev_gadget.spacy

#### 练习：检测模型

让我们来看看模型在未出现过的新数据上表现如何！为了节省时间，我们已经在一些文本上面 训练好了一个带有标签`"GADGET"`的流程。这里是一些结果：

| 文本                                                         | 实体                   |
| ------------------------------------------------------------ | ---------------------- |
| 苹果已经开始让iPhone 8和iPhone X变得越来越慢了，怎么办       | `(iPhone 8, iPhone X)` |
| 我终于明白iPhone X的“刘海”是干嘛的了                         | `(iPhone X,)`          |
| 关于Samsung Galaxy S9你需要了解的一切                        | `(Samsung Galaxy,)`    |
| 想要比较不同的iPad型号？这里是2020年所有的产品线对比。       | `(iPad,)`              |
| iPhone 8和iPhone 8 Plus是苹果公司设计、研发和销售的智能手机  | `(iPhone 8, iPhone 8)` |
| 那个型号ipad是最便宜的，尤其是ipad pro里面的？？             | `(ipad, ipad)`         |
| Samsung Galaxy是三星电子公司设计、生产并退出市场的一系列移动计算设备 | `(Samsung Galaxy,)`    |

在模型从文本中抽取的所有实体中，**有多少实体模型的判断是正确的**？ 注意如果实体的跨度span不完整的话也被认为是一个错误！

小技巧：数一数模型 *应该* 抽取出的实体数目，然后数一数模型 *实际中* 抽取正确的实体数目， 把后者除以前者也就是全部正确实体的数目就可以得到准确率。

1. 45%
2. 60%
3. 70%
4. 90%

**回答：** 

在我们的测试数据上模型抽取的准确率是70%

## 训练spaCy模型的最佳实践

当我们开始跑自己实验的时候，我们可能发现很多东西并不像我们想象的那样 运作完美。这是很正常的。

训练模型是一个迭代的过程，我们需要试很多不同的东西来看怎样才能有最好的结果。

本节课中，我们分享一些训练自己模型的最佳实践和需要铭记在心的要点。

让我们来看看可能遇到的一些问题。

#### 问题一：模型“忘记”了东西

统计模型可以学会很多东西，但它们也会忘记东西。

如果我们用新的数据更新已有模型，特别是更新一些新的标注，模型可能会过拟合，针对新的 例子做出了 *过多* 调整。

举个例子，我们想用"WEBSITE"的例子来更新模型，结果模型就可能“忘记”了之前本来可以预测 正确的诸如"PERSON"这样的类别。

这也被称作“灾难性遗忘”的问题。

- 已有的模型可能会在新数据上过拟合
  - 举个例子：如果我们只是想给模型更新一个`"WEBSITE"`的类别，模型有可能会“忘记” 之前`"PERSON"`这个类别。
- 这也被称为“灾难性遗忘”的问题。

##### 解决方法一：将之前的正确预测结果混合进来

要预防灾难性遗忘问题，我们要确保总是在训练数据中混入一些之前模型预测 正确的例子。

如果我们要训练一个新的类别`"WEBSITE"`，我们就也要把旧的类别`"PERSON"`的例子 包括进来。

spaCy可以帮我们做到这点。我们可以在数据上先跑已有的模型，抽取我们感兴趣的实体， 然后把结果做为新增的训练例子。

然后我们可以把这些例子与我们已有的数据混合起来，用所有类别的标注数据来更新模型。

- 举个例子，我们要训练`"WEBSITE"`，但我们也把`"PERSON"`的例子加进来。
- 在数据上跑已有的spaCy模型然后抽取所有其它相关的实体。

#### 问题二：模型不能学会所有东西

另一个常见的问题是我们的模型不见得就能学到我们想让它学到的东西。

spaCy的模型是基于本地语境作出预测，比如对命名实体来说，目标词符周围的词是最重要的。

如果基于语境本身就很难做出判断，那模型也很难学得会。

标签种类最好前后一致，也不要过于特殊。

举个例子，可能模型很难学会基于语境预测一段文本是成人服饰还是儿童服饰，但如果 只是预测“服饰”模型就能表现得好很多。

- spaCy的模型基于**本地语境**作出预测。
- 如果基于语境本身就很难做出判断，那模型也很难学得会
- 标签种类最好前后一致，也不要过于特殊
  - 比如，`"CLOTHING"`这个类别就要比`"ADULT_CLOTHING"`和`"CHILDRENS_CLOTHING"`更 好。

##### 解决方法二：仔细计划标签种类

我们在训练和更新模型之前，值得先暂停一下，先计划好我们的标签内容。

尽量选择那些能从本地语境中反映出来的类别，而且越通用越好。

我们可以最后再添加基于规则的系统把通用标签转化为特定种类。

通用标签如“服饰”或者“乐队”更容易被标注也更容易被模型学习。

- 选择那些能从本地语境中反映出来的类别
- 更通用的标签要好过更特定的标签
- 我们可以用规则把通用标签转化为特定种类

**不好的做法：**

```python
LABELS = ["ADULT_SHOES", "CHILDRENS_SHOES", "BANDS_I_LIKE"]
```

**好的做法：**

```python
LABELS = ["CLOTHING", "BAND"]
```


#### 练习：好数据vs烂数据

这是一段摘抄，来自于一个训练集试图在旅行者的评论中标注实体类型 `TOURIST_DESTINATION`（游客目的地）。


In [ ]:
doc1 = nlp("我去年去了西安，那里的城墙很壮观！")
doc1.ents = [Span(doc1, 5, 7, label="TOURIST_DESTINATION")]

doc2 = nlp("人一辈子一定要去一趟巴黎，但那里的埃菲尔铁塔有点无聊。")
doc2.ents = [Span(doc2, 10, 12, label="TOURIST_DESTINATION")]

doc3 = nlp("深圳也有个巴黎的埃菲尔铁塔，哈哈哈")
doc3.ents = []

doc4 = nlp("北京很适合暑假去：长城、故宫，还有各种好吃的小吃！")
doc4.ents = [Span(doc4, 0, 2, label="TOURIST_DESTINATION")]

##### 第一部分

为什么这段数据和标注方法有问题？

1. 一个地方是不是游客目的地是一个主观看法而不是客观绝对的，所以实体识别器很难学习到。
2. 埃菲尔铁塔为了保持一致也应该被标注为游客目的地，不然扰乱模型的判断。
3. 像拼写错误的'爸黎'这种非常罕见的词库以外的词就不应该被标注为实体。

提交

1. **正确！** 

   一个更好的方法应该是只标注`"GPE"`（地理政治实体）或者是`"LOCATION"`（位置实体），然后用基于规则的系统 来判断这个实体在语境中是不是游客目的地，比如我们可以在知识库中寻找该实体类别或者在旅行百科 中查询这些实体。

2. **正确！** 

   一个更好的方法应该是只标注`"GPE"`（地理政治实体）或者是`"LOCATION"`（位置实体），然后用基于规则的系统 来判断这个实体在语境中是不是游客目的地，比如我们可以在知识库中寻找该实体类别或者在旅行百科 中查询这些实体。

3. **不正确。** 

   就算是不常见的或者拼写错误的词也是可以被标注为实体的。实际上基于语境来判断拼写错误的文本 的类别恰恰是统计实体识别模型的一大优势。


##### 第二部分

- 重写`doc.ents`使其跨度span的标签为`"GPE"`（城市、州省、国家）而非`"TOURIST_DESTINATION"`。
- 别忘了添加那些数据中本来未被标注为`"GPE"`的实体的跨度span。


In [ ]:
import spacy
from spacy.tokens import Span

nlp = spacy.blank("zh")

doc1 = nlp("我去年去了西安，那里的城墙很壮观！")
doc1.ents = [Span(doc1, 5, 7, label="GPE")]

doc2 = nlp("人一辈子一定要去一趟巴黎，但那里的埃菲尔铁塔有点无趣。")
doc2.ents = [Span(doc2, 10, 12, label="GPE")]

doc3 = nlp("深圳也有个巴黎的埃菲尔铁塔，哈哈哈")
doc3.ents = [Span(doc3, 0, 2, label="GPE"), Span(doc3, 5, 7, label="GPE")]

doc4 = nlp("北京很适合暑假去：长城、故宫，还有各种好吃的小吃！")
doc4.ents = [Span(doc4, 0, 2, label="GPE")]

#### 练习：训练多个标签

这里是某个数据集的一个样品，我们创建它来训练一个新的实体种类`"WEBSITE"`。 原始的数据集包含了几千个句子。这个练习中我们要手动做标注。实际工作中我们 很可能想要用一些标注工具来自动化这一步，比如[Brat](http://brat.nlplab.org/)， 一个很流行的开源方案，或者[Prodigy](https://prodi.gy/)，我们自己开发的整合 了spaCy的标注工具。


##### 第一部分

- 完成数据中所有`"WEBSITE"`实体的位置参数。


In [ ]:
import spacy
from spacy.tokens import Span

nlp = spacy.blank("zh")

doc1 = nlp("哔哩哔哩与阿里巴巴合作为博主们建立社群")
doc1.ents = [
    Span(doc1, 0, 4, label="WEBSITE"),
    Span(doc1, 5, 9, label="WEBSITE"),
]

doc2 = nlp("李子柒打破了Youtube的记录")
doc2.ents = [Span(doc2, 6, 13, label="WEBSITE")]

doc3 = nlp("阿里巴巴的创始人马云提供了一千万购物优惠券")
doc3.ents = [Span(doc3, 0, 4, label="WEBSITE")]

# And so on...

##### 第二部分

我们已经用刚才标注好的数据和其它几千个类似的例子训练了一个模型。训练完成之后，这个模型 对`"WEBSITE"`的抽取表现很好，但却识别不了`"PERSON"`了。这是怎么回事？

1. 对模型来说很难学习到如`"PERSON"`和`"WEBSITE"`这样不同的类别。
2. 训练数据中没有任何`"PERSON"`的例子了，所以模型学习到这个标签本身是错误的。
3. 我们需要返回模型超参数来让两种实体类别都被识别出来。

回答：

1. **不正确。** 

   让模型学习到不同类别是完全可能的。比如spaCy的预训练英文模型就可以识别人名、组织名或者 百分数。

2. **正确！** 

   如果`"PERSON"`实体在训练数据中出现但并未被标注，模型就会学到这些实体不应该被抽取出来。 类似的如果一个已有的实体类别没有出现在训练数据中，模型就会 ”忘记” 它而停止抽取。

3. **不正确。** 

   超参数确实对模型准确度有影响，但不是这里的问题所在。


##### 第三部分

- 更新训练数据，加入对`"PERSON"`实体”李子柒”和”马云”的标注。


In [ ]:
import spacy
from spacy.tokens import Span

nlp = spacy.blank("zh")

doc1 = nlp("哔哩哔哩与阿里巴巴合作为博主们建立社群")
doc1.ents = [
    Span(doc1, 0, 4, label="WEBSITE"),
    Span(doc1, 5, 9, label="WEBSITE"),
]

doc2 = nlp("李子柒打破了Youtube的记录")
doc2.ents = [Span(doc2, 0, 3, label="PERSON"), Span(doc2, 6, 13, label="WEBSITE")]

doc3 = nlp("阿里巴巴的创始人马云提供了一千万购物优惠券")
doc3.ents = [Span(doc3, 0, 4, label="WEBSITE"), Span(doc3, 8, 10, label="PERSON")]

# And so on...

## 总结